In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from os import walk, getcwd
#import h5py

##All npy files are stored in the train_data folder:
mypath = "./train_data/"
txt_name_list = []
for (dirpath, dirnames, filenames) in walk(mypath):
    if filenames != '.DS_Store':       ##Ugh mac junk
        txt_name_list.extend(filenames)
        break


In [2]:
len(txt_name_list)

345

In [3]:
x_train = []
x_test = []
y_train = []
y_test = []
xtotal = []
ytotal = []
slice_train = int(80000/len(txt_name_list))  ###Setting value to be 80000 for the final dataset
i = 0
seed = np.random.randint(1, 10e6)

In [4]:
##Creates test/train split with quickdraw data
for txt_name in txt_name_list:
    txt_path = mypath + txt_name
    x = np.load(txt_path)
    x = x.astype('float32') / 255.    ##scale images
    y = [i] * len(x)  
    np.random.seed(seed)
    np.random.shuffle(x)
    np.random.seed(seed)
    np.random.shuffle(y)
    x = x[:slice_train]
    y = y[:slice_train]
    if i != 0: 
        xtotal = np.concatenate((x,xtotal), axis=0)
        ytotal = np.concatenate((y,ytotal), axis=0)
    else:
        xtotal = x
        ytotal = y
    i += 1
x_train, x_test, y_train, y_test = train_test_split(xtotal, ytotal, test_size=0.2, random_state=42)

In [5]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np

Using TensorFlow backend.


In [6]:
batch_size = 128
num_classes = len(txt_name_list)
epochs = 25

# to show
print('Num classes: ', num_classes)

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
# (x_train, y_train), (x_test, y_test) = mnist.load_data()

print('\noriginal x_train shape:', x_train.shape)
print('original x_test shape:', x_test.shape)


Num classes:  345

original x_train shape: (63756, 784)
original x_test shape: (15939, 784)


In [7]:
# Based on previous shape
x_train = x_train.reshape(x_train.shape[0],28,28,1)
x_test = x_test.reshape(x_test.shape[0],28,28,1)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (63756, 28, 28, 1)
63756 train samples
15939 test samples


In [8]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(28,28,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Instructions for updating:
Use tf.cast instead.
Train on 63756 samples, validate on 15939 samples
Epoch 1/25
63756/63756 [==============================] - 199s 3ms/step - loss: 5.2128 - acc: 0.0454 - val_loss: 4.0850 - val_acc: 0.1739
Epoch 2/25
63756/63756 [==============================] - 206s 3ms/step - loss: 4.1589 - acc: 0.1552 - val_loss: 3.5085 - val_acc: 0.2820
Epoch 3/25
63756/63756 [==============================] - 193s 3ms/step - loss: 3.6721 - acc: 0.2258 - val_loss: 3.0812 - val_acc: 0.3421
Epoch 4/25
63756/63756 [==============================] - 193s 3ms/step - loss: 3.4133 - acc: 0.2667 - val_loss: 2.9547 - val_acc: 0.3655
Epoch 5/25
63756/63756 [==============================] - 192s 3ms/step - loss: 3.2312 - acc: 0.2986 - val_loss: 2.8097 - val_acc: 0.3815
Epoch 6/25
63756/63756 [==============================] - 192s 3ms/step - loss: 3.1074 - acc: 0.3194 - val_loss: 2.7446 - val_acc: 0.4075
Epoch 7/25
63756/63756 [==============================] - 191s 3ms/step - 

In [11]:
# More epochs
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=25,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 63756 samples, validate on 15939 samples
Epoch 1/25
63756/63756 [==============================] - 196s 3ms/step - loss: 2.1277 - acc: 0.4908 - val_loss: 2.6702 - val_acc: 0.4616
Epoch 2/25
63756/63756 [==============================] - 197s 3ms/step - loss: 2.1094 - acc: 0.4931 - val_loss: 2.5161 - val_acc: 0.4705
Epoch 3/25
63756/63756 [==============================] - 198s 3ms/step - loss: 2.1006 - acc: 0.4975 - val_loss: 2.5359 - val_acc: 0.4448
Epoch 4/25
63756/63756 [==============================] - 196s 3ms/step - loss: 2.0888 - acc: 0.4982 - val_loss: 2.4050 - val_acc: 0.4631
Epoch 5/25
63756/63756 [==============================] - 195s 3ms/step - loss: 2.0710 - acc: 0.4999 - val_loss: 2.4827 - val_acc: 0.4608
Epoch 6/25
63756/63756 [==============================] - 196s 3ms/step - loss: 2.0742 - acc: 0.5002 - val_loss: 2.5399 - val_acc: 0.4493
Epoch 7/25
63756/63756 [==============================] - 196s 3ms/step - loss: 2.0478 - acc: 0.5049 - val_loss: 2.4220 - 

In [12]:
from keras.models import load_model
model.save('quickdraw_all.h5')